In [ ]:
import networkx as nx
import pandas as pd
from networkx.algorithms import bipartite


employees = set([
    'Pablo',
    'Lee',
    'Georgia',
    'Vincent',
    'Andy',
    'Frida',
    'Joan',
    'Claude'
])


movies = set([
    'The Shawshank Redemption',
    'Forrest Gump',
    'The Matrix',
    'Anaconda',
    'The Social Network',
    'The Godfather',
    'Monty Python and the Holy Grail',
    'Snakes on a Plane',
    'Kung Fu Panda',
    'The Dark Knight',
    'Mean Girls'
])


def plot_graph(G, weight_name=None):
    import matplotlib.pyplot as plt

    plt.figure()
    pos = nx.spring_layout(G)

    if weight_name:
        labels = nx.get_edge_attributes(G, weight_name)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
        nx.draw_networkx(G, pos)
    else:
        nx.draw_networkx(G, pos)

In [ ]:
# 1ο ερώτημα.
g1_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t',
                    header=None, skiprows=1, names=['Employees', 'Movies'])
g1 = nx.from_pandas_edgelist(g1_df, 'Employees', 'Movies')
plot_graph(g1)

In [ ]:
# 2ο ερώτημα.
g2 = g1
g2.add_nodes_from(employees, bipartite=0, type='employee')
g2.add_nodes_from(movies, bipartite=1, type='movie')
plot_graph(g2)

In [ ]:
# 3o ερώτημα.
g3 = g2
weighted_projection = bipartite.weighted_projected_graph(g3, employees)
plot_graph(weighted_projection, 'weight')

In [ ]:
g4 = pd.DataFrame(weighted_projection.edges(data=True),
                  columns=['Emp1', 'Emp2', 'movies_in_common'])
g4['movies_in_common'] = g4['movies_in_common'].map(lambda x: x['weight'])
g4_duplicate = g4.copy()
g4_duplicate.rename(columns={'Emp1': 'Emp2', 'Emp2': 'Emp1'}, inplace=True)
g4_common_movies = pd.concat([g4, g4_duplicate], ignore_index=True)
g4_employee_rel = pd.read_csv('Employee_Relationships.txt', sep='\t', header=None, names=[
                              'Emp1', 'Emp2', 'Relationship'])
g4_merged = pd.merge(g4_employee_rel, g4_common_movies, how='left')
g4_merged['movies_in_common'].fillna(value=0, inplace=True)
correlation_score = g4_merged['movies_in_common'].corr(
    g4_merged['Relationship'])

print(correlation_score)